In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
from decimal import *
import numpy as np
import pandas as pd
import sys
import requests
from dotenv import load_dotenv
import os, os.path
from requests.adapters import HTTPAdapter, Retry
from pathlib import Path  # Python 3.6+ only
import xlsxwriter
from datetime import date

#loads env vars from local .env file
def load_env():
    #set .env path and load 
    env_path = Path('.') / '.env'
    load_dotenv(dotenv_path=env_path)

#creates generic HTTP session/request
def create_HTTP_session():
    #create a requests session to get/post; HTTPS with HTTPAdapter
    s = requests.Session()
    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    return s

#uses session/request to get authentication response JSON for springcm
def get_springcm_authentication(session, client_id, client_secret):
    # Get access token and API base URL
    response = session.post(
        "https://auth.springcm.com/api/v201606/apiuser",
        data={"client_id": client_id, "client_secret": client_secret},
    )
    authentication_response = response.json()
    return authentication_response

#downloads single scpringcm file by ID, example call: download_springcm_file_by_ID(os.environ.get('SERVICES_HOURS'), auth_response, 'Services by Hour.csv', old_path, s)
def download_springcm_file_by_ID(ID, authentication_response, dest_document_name, dest_folder_path, session):
    response = session.get(
        "https://apidownloadna11.springcm.com/v201411/documents/" + ID,
        headers={
            "Authorization": "Bearer " + authentication_response["access_token"],
        },
    )
    os.makedirs(os.path.dirname(dest_folder_path), exist_ok=True)
    with open(dest_folder_path + dest_document_name, "wb") as f:
        f.write(response.content)

#downloads all files in a given springcm folder, example call: download_springcm_all_files(auth_response, './agstarData/', s, "/AgStar Data/New AgStar Reports") LEAVE OUT "Wegis and/& Young"
def download_springcm_all_files(authentication_response, dest_folder_path, session, springcm_folder_path):
    # Get folder paths
    response = session.get(
        "https://apina11.springcm.com/v201411/folders",
        headers={
            "Authorization": "Bearer " + authentication_response["access_token"],
        },
        params={
            "path": springcm_folder_path,
            "expand": "documents,folders",
            "limit": 100,
        },
    )
    folder_response = response.json()

    # Grab all documents in folder
    document_list = {}
    for document in folder_response['Documents']['Items']:
        document_list.update({document['Name']: document['DownloadDocumentHref']})
    for document_name in document_list:
        document_id = document_list[document_name]
        response = session.get(
            document_id,
            headers={
            "Authorization": "Bearer " + authentication_response['access_token'],
        }
        )
        os.makedirs(os.path.dirname(dest_folder_path), exist_ok=True)
        with open(dest_folder_path + document_name, "wb") as f:
            f.write(response.content)

#assigns a row number to a dataframe, returns dataframe
def assignRowNum(dataframe):
    
    w = Window().orderBy(lit('A'))
    dataframe = dataframe.withColumn('row_num', row_number().over(w))
    
    return dataframe

#fills oldColumnName down/forwards, gives newColumnName
def fillColumnDown(dataframe, partitionBy, orderBy, oldColumnName, newColumnName):
        
        window = (
        Window
        .partitionBy(partitionBy)
        .orderBy(orderBy)
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)
      )
        
        dataframe = dataframe.withColumn(newColumnName, last(oldColumnName, ignorenulls=True).over(window)) 
       
        
        return dataframe

#splits columns by pipe with custom parameters
def customPipeSplit(dataframe, price, units, service, percent):   
    dataframe = dataframe.withColumn('split_table',
                        explode(
                        arrays_zip(
                        split(col(units), '\\|').alias("units_arr")
                       ,split(col(service), '\\|').alias("service_arr")
                       ,split(col(price), '\\|').alias("price_arr")
                       ,split(col(percent), '\\|').alias("percent_arr"))))

    dataframe = dataframe.withColumn('PRICE', col('split_table')['price_arr']) \
          .withColumn('PERCENT', col('split_table')['percent_arr']) \
          .withColumn('SERVICE', col('split_table')['service_arr']) \
          .withColumn('BILLABLE UNITS', col('split_table')['units_arr']) \
          .drop('split_table')
    return dataframe

#adds a row of all spaces after each group
def addSpaceAfterGroupPandas(df,group):
    # reset index 
    df = df.reset_index(drop = True)

    # find the border idx of InvoiceNumber
    idx_list = df.drop_duplicates(group, keep='last').index

    # use the border idx, create an empty df, and concat to the origin df, then sort the index
    df_append = pd.DataFrame('', index = idx_list, columns = df.columns)
    df = pd.concat([df,df_append]).sort_index()                 
    
    
    #reset index and drop unused columns
    df = df.reset_index()
    df = df.sort_values(by=['index', group],ascending = [True, True])  
    df.drop([group, 'index'], axis=1, inplace=True)
    df = df.reset_index(drop = True)
    return df

#replaces prices on tickets with prices in services
def replace_prices(dataframe, prices_data):
    #drop row number
    dataframe = dataframe.drop('row_num')

    #cast as floats, round billable units to 2 decimal places
    dataframe = dataframe.withColumn("PRICE",col("PRICE").cast('Decimal(20,2)')) \
           .withColumn("BLANK",lit('')) \
           .withColumn("PERCENT",col("PERCENT").cast('Decimal(20,2)')) \
           .withColumn("BILLABLE UNITS",   col("BILLABLE UNITS").cast('Decimal(20,2)')) \
           .withColumn("BILLABLE UNITS",   (col("BILLABLE UNITS")*(col("PERCENT")/100.00)).cast(DecimalType(20,2)))  \



    #joining services prices and renaming columns
    dataframe = dataframe.join(prices,(dataframe.SERVICE == prices_data.SERVICE_P) & (dataframe.UNITS == prices_data.UNITS_P)) \
           .drop('SERVICE', 'PRICE','UNITS') \
           .withColumnRenamed("PRICE_P", "PRICE") \
           .withColumnRenamed("SERVICE_P", "SERVICE") \
           .withColumnRenamed("UNITS_P", "UNITS") 
    return dataframe

#union price by acre/hour
def prices_by_unit_union(price_by_acre, price_by_hour):
    #--------------------------------
    #rename Services by Acre AND Services by Hour to column match
    price_by_acre = price_by_acre.withColumn("PRICE",col("Price per Acre").cast('Decimal(20,2)')) \
                             .withColumn('UNITS', lit('Acres')) \
                             .withColumnRenamed('Service by Acre', 'SERVICE') \
                            .drop('Service by Acre', "Price per Acre")

    price_by_hour = price_by_hour.withColumn("PRICE",col("Price per Hour").cast('Decimal(20,2)'))    \
                             .withColumn('UNITS', lit('Hours')) \
                             .withColumnRenamed('Service by Hour', 'SERVICE') \
                             .drop('Service by Hour', "Price per Hour")  

    # Union Services by Acre AND Services by Hour   
    prices = price_by_hour.union(price_by_acre)

    #renaming columns to avoid conflicts below
    prices = prices.withColumnRenamed("PRICE", "PRICE_P") \
               .withColumnRenamed("SERVICE", "SERVICE_P") \
               .withColumnRenamed("UNITS", "UNITS_P")
    return prices

#unions tickets by acre and tickets by hour, pipe split, fill down, replace prices with services
def tickets_union(service_by_acre, service_by_hour, prices):
        #drop unused columns 
    service_by_acre = service_by_acre.drop('RANCH MANAGER', 'DOCUMENT NAME', 'DATE', 'STATUS', 'REC NUMBER','NOTES')

    #rename columns, add literals 
    service_by_acre = service_by_acre.withColumnRenamed('SERVICE BY ACRE', 'SERVICE') \
                                 .withColumnRenamed('ACRES', 'BILLABLE UNITS') \
                                 .withColumnRenamed('PRICE PER ACRE', 'PRICE') \
                                 .withColumn('UNITS', lit('Acres'))
    #drop unused columns 
    service_by_hour = service_by_hour.drop('RANCH MANAGER', 'DOCUMENT NAME', 'DATE', 'STATUS', 'REC NUMBER', 'ACRES', 'NOTES')

    #rename columns , add literals 
    service_by_hour = service_by_hour.withColumnRenamed('SERVICE BY HOUR', 'SERVICE') \
                                 .withColumnRenamed('HOURS', 'BILLABLE UNITS') \
                                 .withColumnRenamed('PRICE PER HOUR', 'PRICE') \
                                 .withColumn('PERCENT', lit('100')) \
                                 .withColumn('UNITS', lit('Hours'))

    #union Service by Hour AND Service by Acre
    tickets = service_by_hour.union(service_by_acre)
    
     #custom split
    tickets = customPipeSplit(tickets, 'PRICE', 'BILLABLE UNITS', 'SERVICE', 'PERCENT')

    #assign row numbers
    tickets = assignRowNum(tickets)

    #fill down on columns
    tickets = fillColumnDown(tickets, 'FIELD TICKET NUMBER', 'row_num', 'BILLABLE UNITS', 'BILLABLE UNITS')
    tickets = fillColumnDown(tickets, 'FIELD TICKET NUMBER', 'row_num', 'PRICE', 'PRICE')
    tickets = fillColumnDown(tickets, 'FIELD TICKET NUMBER', 'row_num', 'PERCENT', 'PERCENT')
    tickets = fillColumnDown(tickets, 'FIELD TICKET NUMBER', 'row_num', 'SERVICE', 'SERVICE')

    #replace prices in tickets with prices from services
    tickets = replace_prices(tickets, prices)
    
    return tickets

#create FT to Archive file
def field_ticket_archive(tickets, folder_path):
       
    #create FT to Archive CSV
    to_archive = assignRowNum(tickets)
    to_archive = to_archive.withColumnRenamed('row_num', 'ROWID')
    to_archive.toPandas().to_csv(folder_path + 'Service for AgStar - FT to Archive.csv',header = True, index = False)
    
#creates agstar report, requires tempview sql from spark named field _tickets
def create_agstar_csv(agstar_report_path):  
    #select agstar report                   
    agstar = spark.sql("SELECT `CUSTOMER NUMBER`                                                                    \
                       , DENSE_RANK() OVER (ORDER BY COMPANY ASC)AS InvoiceNumber                                   \
                       , DATE_FORMAT(DATE_ADD(LAST_DAY(ADD_MONTHS(CURRENT_DATE(), -1)), 1), 'M/d/yyyy') AS DATE1                    \
                       , DENSE_RANK() OVER (PARTITION BY COMPANY ORDER BY SERVICE ASC, UNITS ASC) AS LineSeq        \
                       , ACCOUNT                                                                                    \
                       , ROUND(SUM(`BILLABLE UNITS`)*PRICE,2) AS TOTAL                                              \
                       , BLANK                                                                                      \
                       , DATE_FORMAT(LAST_DAY(CURRENT_DATE()), 'M/d/yyyy') AS DATE2                  \
                       , SERVICE                                                                                    \
                       , SUM(`BILLABLE UNITS`) AS `BILLABLE UNITS`                                                  \
                       , UNITS                                                                                      \
                       , PRICE                                                                                      \
                       , 'N'                                                                                        \
                       , `COST CENTER`                                                                              \
                       , '0'                                                                                        \
                        FROM field_tickets                                                                          \
                        GROUP BY  COMPANY                                                                           \
                                , SERVICE                                                                           \
                                , `CUSTOMER NUMBER`                                                                 \
                                , ACCOUNT                                                                           \
                                , UNITS                                                                             \
                                , `COST CENTER`                                                                     \
                                , PRICE                                                                             \
                                , BLANK                                                                             \
                        ORDER BY  COMPANY                                                                           \
                                , SERVICE ")
    #convert to pandas for transformation/writes csv
    agstar.toPandas().to_csv(agstar_report_path, header = False, index = False)

#creates xlsx file customer detail report, requires tempview from spark named field _tickets
def create_customer_detail_xlsx(customer_detail_report_path):
    
    
        #select customer detail report
    customer_detail = spark.sql("SELECT COMPANY                                                                     \
                               , `FIELD TICKET NUMBER` AS `Ticket Number`                                          \
                               , FIELD  AS Field                                                                   \
                               , `BILLABLE UNITS`  AS `Acres/Hours`                                                \
                               , SERVICE AS Service                                                                \
                               , PRICE AS Price                                                                    \
                               , `BILLABLE UNITS`*PRICE AS `Line Total`                                            \
                               , DENSE_RANK() OVER (ORDER BY COMPANY ASC) AS InvoiceNumber                         \
                               , DATE_FORMAT(DATE_ADD(LAST_DAY(ADD_MONTHS(CURRENT_DATE(), -1)), 1), 'M/d/yyyy') AS DATE1                    \
                               , DATE_FORMAT(LAST_DAY(CURRENT_DATE()), 'M/d/yyyy') AS DATE2                  \
                               FROM field_tickets                                                                  \
                               ORDER BY COMPANY                                                                    \
                                       ,SERVICE ")
    #convert to pandas for transformation
    customer_detail = customer_detail.toPandas()
    
    
    workbook = xlsxwriter.Workbook(customer_detail_report_path)

    bold_centered = workbook.add_format({'bold': 1})
    bold_centered.set_align('vcenter')
    bold_centered.set_align('hcenter')
    bold1 = workbook.add_format({'bold': 1})
    bold1.set_bottom()
    bold1.set_top()
    bold2 = workbook.add_format({'bold': 1,'num_format': '$#,##0.00'})
    bold2.set_bottom(2)
    bold2.set_top()
    center = workbook.add_format()
    center.set_align('vcenter')
    format1 = workbook.add_format({'num_format': '#,##0.00'})
    format1.set_pattern(1)
    format1.set_bg_color('#D9D9D9')
    format2 = workbook.add_format({'num_format': '#,##0.00'})
    
    for i in range(customer_detail['InvoiceNumber'].max()):
        df1 = customer_detail[customer_detail['InvoiceNumber'] == i+1]
        company = df1['COMPANY'].iloc[0]
        ws = workbook.add_worksheet(company)
        if '&' in company:
            string = company.split('&')
            company = string[0] + '&&' + string[1]
        ws.set_header('&C&18&"Calibri,Bold"' + company)
        ws.write('A1', 'Invoice #' + str(i+1), bold_centered)
        ws.write('D1', 'Date Range:', bold_centered)
        ws.write('E1', df1['DATE1'].iloc[0],center)
        ws.write('F1', df1['DATE2'].iloc[0],center)
        ws.write('A2', 'Ticket Number', bold1)
        ws.write('B2', 'Field', bold1)
        ws.write('C2', 'Acres/Hours', bold1)
        ws.write('D2', 'Service', bold1)
        ws.write('E2', 'Price', bold1)
        ws.write('F2', 'Line Total', bold1)
        ws.set_column(0, 0, 15.43)
        ws.set_column(2, 2, 10)
        ws.set_column(3, 3, 15.14)
        ws.set_column(4, 4, 12.57)
        ws.set_column(5, 5, 14.29)
        ws.set_row(0, 52.5)
        ws.set_page_view()
        df1 = df1.reset_index(drop = True)
        for i in range(0 ,df1.last_valid_index()+1, 2):
            ws.write('A'+ str(i+3), df1['Ticket Number'].iloc[i], format1)
            ws.write('B'+ str(i+3), df1['Field'].iloc[i], format1)
            ws.write('C'+ str(i+3), df1['Acres/Hours'].iloc[i], format1)
            ws.write('D'+ str(i+3), df1['Service'].iloc[i], format1)
            ws.write('E'+ str(i+3), df1['Price'].iloc[i], format1)
            ws.write('F'+ str(i+3), df1['Line Total'].iloc[i], format1)

            if(i == df1.last_valid_index()):
                break
            ws.write('A'+ str(i+4), df1['Ticket Number'].iloc[i+1], format2)
            ws.write('B'+ str(i+4), df1['Field'].iloc[i+1], format2)
            ws.write('C'+ str(i+4), df1['Acres/Hours'].iloc[i+1], format2)
            ws.write('D'+ str(i+4), df1['Service'].iloc[i+1], format2)
            ws.write('E'+ str(i+4), df1['Price'].iloc[i+1], format2)
            ws.write('F'+ str(i+4), df1['Line Total'].iloc[i+1], format2)

        ws.write('A'+ str(df1.last_valid_index()+4) , 'Total:', bold2)
        ws.write('B'+ str(df1.last_valid_index()+4) , '', bold2)
        ws.write('C'+ str(df1.last_valid_index()+4) , '', bold2)
        ws.write('D'+ str(df1.last_valid_index()+4) , '', bold2)
        ws.write('E'+ str(df1.last_valid_index()+4) , '', bold2)
        ws.write('F'+ str(df1.last_valid_index()+4) , '=SUM(F3:F' + str(df1.last_valid_index()+3) + ')', bold2)
    workbook.close()
##################################################################################
#load env vars; access via os.environ.get('VAR')
load_env()

#create HTTP session with retries etc
s = create_HTTP_session()

#get auth info from springcm
auth_response = get_springcm_authentication(s, os.environ.get('CLIENT_ID'), os.environ.get('CLIENT_SECRET'))

#date info
today = date.today()
today = today.strftime("%b-%d-%Y")

#assign folder/file paths to local OLD/NEW
old_folder_path = './agstarDataOld/'
new_folder_path = './agstarDataNew/'
services_acre = 'Services by Acre.csv'
services_hour = 'Services by Hour.csv'
tickets_acre = 'Service By Acre for AgStar Import.csv'
tickets_hour = 'Service By Hour for AgStar Import.csv'
customer_detail_report_path = new_folder_path + 'Customer Detail Report ' + today + '.xlsx'
agstar_report_path = new_folder_path + 'AgStar Report ' + today + '.csv'

#make sure old and new folders are created
os.makedirs(os.path.dirname(new_folder_path), exist_ok=True)
os.makedirs(os.path.dirname(old_folder_path), exist_ok=True)

#Download Services by Acre/Hour.csv    
download_springcm_file_by_ID(os.environ.get('SERVICES_ACRES'), auth_response, services_acre, old_folder_path, s)
download_springcm_file_by_ID(os.environ.get('SERVICES_HOURS'), auth_response, services_hour, old_folder_path, s)

#Download Service By Hour/Acre for AgStar Import.csv   
download_springcm_file_by_ID(os.environ.get('TICKETS_ACRES'), auth_response, tickets_acre, old_folder_path, s)
download_springcm_file_by_ID(os.environ.get('TICKETS_HOURS'), auth_response, tickets_hour, old_folder_path, s)

#creating spark session
spark = SparkSession.builder.master("local[1]").appName("CRoberts").getOrCreate()  

#open Service/Prices by Acres AND Service/Prices by Hours 
price_by_acre =  spark.read.option("header",True).option("multiline", "true").csv(old_folder_path+services_acre)
price_by_hour =  spark.read.option("header",True).option("multiline", "true").csv(old_folder_path+services_hour) 
#open Service by Hour/Acre
service_by_hour =  spark.read.option("header",True).option("multiline", "true").csv(old_folder_path+tickets_hour)
service_by_acre =  spark.read.option("header",True).option("multiline", "true").csv(old_folder_path+tickets_acre)

#union all service prices together with units
prices = prices_by_unit_union(price_by_acre, price_by_hour)

#union field tickets, and replace with cprices with corrected prices 
tickets = tickets_union(service_by_acre, service_by_hour, prices)

#create FT to Archive file
field_ticket_archive(tickets, new_folder_path)

#create ***GLOBAL*** sql queryable view
tickets.createOrReplaceTempView("field_tickets")
#create agstar/customer detail files
create_agstar_csv(agstar_report_path)
create_customer_detail_xlsx(customer_detail_report_path)

print('Success!')



Success!


In [7]:
import requests
from dotenv import load_dotenv
import os, os.path
from requests.adapters import HTTPAdapter, Retry


load_dotenv()

# OR, the same with increased verbosity
load_dotenv(verbose=True)

# OR, explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

# Print variable FOO
print(os.environ.get('CLIENT_ID')) # Returns 'BAR'

s = requests.Session()
retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))

# Get access token and API base URL
response = s.post(
    "https://auth.springcm.com/api/v201606/apiuser",
    data={"client_id": os.environ.get('CLIENT_ID'), "client_secret": os.environ.get('CLIENT_SECRET')},
)
auth_response = response.json()
# Get folder paths
response = s.get(
    "https://apina11.springcm.com/v201411/folders",
    headers={
        "Authorization": "Bearer " + auth_response["access_token"],
    },
    params={
        "path": "/AgStar Data/New AgStar Reports",
        "expand": "documents,folders",
        "limit": 100,
    },
)
folder_response = response.json()


# Grab all documents in folder
document_list = {}
for document in folder_response['Documents']['Items']:
    document_list.update({document['Name']: document['DownloadDocumentHref']})
print(document_list)
for document_name in document_list:
    document_id = document_list[document_name]
    response = requests.get(
        document_id,
        headers={
        "Authorization": "Bearer " + auth_response['access_token'],
    }
    )
    os.makedirs(os.path.dirname('./agstarData/'), exist_ok=True)
    with open('./agstarData/' + document_name, "wb") as f:
        f.write(response.content)

e67384ef-6bdf-49db-840c-c1cf6f44c36c
{'Service By Hour for AgStar Import.csv': 'https://apidownloadna11.springcm.com/v201411/documents/270ae122-bdb9-ec11-9c48-d89d6716196d', 'Service By Acre for AgStar Import.csv': 'https://apidownloadna11.springcm.com/v201411/documents/630ae122-bdb9-ec11-9c48-d89d6716196d'}


In [2]:
# Example from https://pypi.org/project/python-dotenv/
from dotenv import load_dotenv
import os, os.path
load_dotenv()

# OR, the same with increased verbosity
load_dotenv(verbose=True)

# OR, explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

# Print variable FOO
print(os.environ.get('CLIENT_ID')) # Returns 'BAR'

e67384ef-6bdf-49db-840c-c1cf6f44c36c


In [2]:
import requests
from dotenv import load_dotenv
import os, os.path
from requests.adapters import HTTPAdapter, Retry
from pathlib import Path  # Python 3.6+ only

#loads env vars from local .env file
def load_env():
    #set .env path and load 
    env_path = Path('.') / '.env'
    load_dotenv(dotenv_path=env_path)

#creates generic HTTP session/request
def create_HTTP_session():
    #create a requests session to get/post; HTTPS with HTTPAdapter
    s = requests.Session()
    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    return s

#uses session/request to get authentication response JSON for springcm
def get_authentication(session, client_id, client_secret):
    # Get access token and API base URL
    response = session.post(
        "https://auth.springcm.com/api/v201606/apiuser",
        data={"client_id": client_id, "client_secret": client_secret},
    )
    authentication_response = response.json()
    return authentication_response

#downloads single scpringcm file by ID, example call: download_springcm_file_by_ID(os.environ.get('SERVICES_HOURS'), auth_response, 'Services by Hour.csv', old_path, s)
def download_springcm_file_by_ID(ID, authentication_response, dest_document_name, dest_folder_path, session):
    response = session.get(
        "https://apidownloadna11.springcm.com/v201411/documents/" + ID,
        headers={
            "Authorization": "Bearer " + authentication_response["access_token"],
        },
    )
    os.makedirs(os.path.dirname(dest_folder_path), exist_ok=True)
    with open(dest_folder_path + dest_document_name, "wb") as f:
        f.write(response.content)

#downloads all files in a given springcm folder, example call: download_springcm_all_files(auth_response, './agstarData/', s, "/AgStar Data/New AgStar Reports") LEAVE OUT "Wegis and/& Young"
def download_springcm_all_files(authentication_response, dest_folder_path, session, dl_folder_path):
    # Get folder paths
    response = session.get(
        "https://apina11.springcm.com/v201411/folders",
        headers={
            "Authorization": "Bearer " + authentication_response["access_token"],
        },
        params={
            "path": dl_folder_path,
            "expand": "documents,folders",
            "limit": 100,
        },
    )
    folder_response = response.json()

    # Grab all documents in folder
    document_list = {}
    for document in folder_response['Documents']['Items']:
        document_list.update({document['Name']: document['DownloadDocumentHref']})
    for document_name in document_list:
        document_id = document_list[document_name]
        response = session.get(
            document_id,
            headers={
            "Authorization": "Bearer " + authentication_response['access_token'],
        }
        )
        os.makedirs(os.path.dirname(dest_folder_path), exist_ok=True)
        with open(dest_folder_path + document_name, "wb") as f:
            f.write(response.content)

################################################################################################################

#load env vars; access via os.environ.get('VAR')
load_env()

#create HTTP session with retries etc
s = create_HTTP_session()

#get auth info from springcm
auth_response = get_authentication(s, os.environ.get('CLIENT_ID'), os.environ.get('CLIENT_SECRET'))

#assign folder paths to local OLD/NEW
old_path = './agstarDataOld/'
new_path = './agstarDataNew/'

#Download Services by Acre/Hour.csv    
download_springcm_file_by_ID(os.environ.get('SERVICES_ACRES'), auth_response, 'Services by Acre.csv', old_path, s)
download_springcm_file_by_ID(os.environ.get('SERVICES_HOURS'), auth_response, 'Services by Hour.csv', old_path, s)

#Download Service By Hour/Acre for AgStar Import.csv   
download_springcm_file_by_ID(os.environ.get('TICKETS_ACRES'), auth_response, 'Service By Acre for AgStar Import.csv', old_path, s)
download_springcm_file_by_ID(os.environ.get('TICKETS_HOURS'), auth_response, 'Service By Hour for AgStar Import.csv', old_path, s)



        
        



In [25]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 5.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 725.4 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
